# read s3 delta table

the following code can be used to read data from data lake table stored on s3 in the delta file format (similar to parquet). 

we will read the dataframe using pyspark.

In [2]:
import os
import ast
import logging
import pprint
import yaml
import json
import time
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.column import *
import datetime as dt
import pandas as pd
import numpy as np

import boto3
import pprint
import botocore.session
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr

with open("creds.json", "r") as g:
    creds = json.load(g)
    g.close()

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    f.close()

pp = pprint.PrettyPrinter(indent = 1)
# print("creds.json keys: ")
# pp.pprint([k for k in creds.keys()])

spark_host = creds["spark_host"]
spark_port = creds["spark_port"]
aws_client = creds["aws_client"]
aws_secret = creds["aws_secret"]
extra_jar_list = creds["extra_jar_list"]
print("imported modules.")

s3 = boto3.client("s3")
print("connected to boto3 clients.")

imported modules.
connected to boto3 clients.


## delta table s3 path

In [3]:
bucket = creds["bucket"]
subreddit = "aws"
filepath = os.path.join("s3a://", bucket, subreddit + "_clean")
print(filepath)

s3a://reddit-streaming-stevenhurwitt-new/aws_clean


## create spark session

In [4]:
try:
    spark = SparkSession.builder.appName("twitter") \
        .master("spark://{}:{}".format(spark_host, spark_port)) \
        .config("spark.executor.memory", "2048m") \
        .config("spark.executor.cores", "2") \
        .config("spark.streaming.concurrentJobs", "8") \
        .config("spark.local.dir", "/opt/workspace/tmp/driver/") \
        .config("spark.worker.dir", "/opt/workspace/tmp/executor/") \
        .config("spark.eventLog.enabled", "true") \
        .config("spark.eventLog.dir", "/opt/workspace/tmp/events/") \
        .config("spark.sql.debug.maxToStringFields", 1000) \
        .config("spark.jars.packages", extra_jar_list) \
        .config("spark.hadoop.fs.s3a.access.key", aws_client) \
        .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.hadoop.fs.s3a.buffer.dir", "/opt/workspace/tmp/blocks") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
        .enableHiveSupport() \
        .getOrCreate()

    sc = spark.sparkContext
    # index = 0
    sc.setLogLevel("WARN")
    # sc.setLocalProperty("spark.scheduler.pool", "pool{}".format(str(index)))
    print("imported modules, created spark.")

except Exception as f:
    print("EXCEPTION: ")

EXCEPTION: 


JAVA_HOME is not set


## read df

In [6]:
try:
    df = spark.read.format("delta").option("header", True).load(filepath)
    # df.show()
    print("read df.")

except Exception as g:
    print("EXCEPTION: {}".format(g))

EXCEPTION: name 'spark' is not defined


## pandas df

In [7]:
df_pandas = df.toPandas()
df_pandas

NameError: name 'df' is not defined

In [ ]:
df_pandas.shape

(74, 103)